In [5]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# variables for the weather api
# hourly and daily needs to be chosen from
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 51.5167,
	"longitude": 9.9167,
	"start_date": "2018-01-01",
	"end_date": "2024-11-01",
	"hourly": ["temperature_2m", "cloud_cover"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(1).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["cloud_cover"] = hourly_cloud_cover

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)


# save file in csv

csv_file = "daily_weather.csv"
hourly_dataframe.to_csv(csv_file, index = False)

print(f"csv FILE saved as '{csv_file}' in the current directory")

Coordinates 51.49384689331055°N 9.945652961730957°E
Elevation 174.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                           date  temperature_2m  cloud_cover
0     2018-01-01 00:00:00+00:00          10.383        100.0
1     2018-01-01 01:00:00+00:00           9.433        100.0
2     2018-01-01 02:00:00+00:00           8.583        100.0
3     2018-01-01 03:00:00+00:00           7.933         66.0
4     2018-01-01 04:00:00+00:00           7.583         93.0
...                         ...             ...          ...
59923 2024-11-01 19:00:00+00:00             NaN          NaN
59924 2024-11-01 20:00:00+00:00             NaN          NaN
59925 2024-11-01 21:00:00+00:00             NaN          NaN
59926 2024-11-01 22:00:00+00:00             NaN          NaN
59927 2024-11-01 23:00:00+00:00             NaN          NaN

[59928 rows x 3 columns]
CSV file saved as 'weather_data.csv' in the current directory.
